# Train index

Will preform training on index, for compress it

In [1]:
!pip install scann

     |████████████████████████████████| 10.6 MB 5.3 MB/s eta 0:00:01    |██████████████████████▌         | 7.4 MB 3.8 MB/s eta 0:00:01
     |████████████████████████████████| 458.3 MB 46 kB/s  eta 0:00:01    |▉                               | 11.2 MB 6.3 MB/s eta 0:01:12     |█                               | 15.2 MB 5.5 MB/s eta 0:01:21     |█▏                              | 16.7 MB 5.5 MB/s eta 0:01:20     |███▍                            | 48.9 MB 5.3 MB/s eta 0:01:18     |███▉                            | 55.2 MB 4.6 MB/s eta 0:01:28     |████▎                           | 61.8 MB 6.5 MB/s eta 0:01:01     |████▌                           | 63.8 MB 6.5 MB/s eta 0:01:01     |███████                         | 99.4 MB 6.3 MB/s eta 0:00:57     |████████                        | 113.6 MB 6.3 MB/s eta 0:00:55     |████████▋                       | 123.1 MB 6.3 MB/s eta 0:00:53     |█████████▊                      | 139.6 MB 6.0 MB/s eta 0:00:54     |██████████▎                     | 146.4 

  Created wheel for clang: filename=clang-5.0-py3-none-any.whl size=30693 sha256=618bad019d15841da1a2664cef1e46c5e6ec6529a173192f4137f9604bda204c
  Stored in directory: /root/.cache/pip/wheels/22/4c/94/0583f60c9c5b6024ed64f290cb2d43b06bb4f75577dc3c93a7
Successfully built clang
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.5.0rc0
    Uninstalling tensorflow-estimator-2.5.0rc0:
      Successfully uninstalled tensorflow-estimator-2.5.0rc0
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.34.1
    Uninstalling grpcio-1.34.1:
      Successfully uninstalled grpcio-1.34.1
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.5.0
    Uninstalling tensorboard-2.5.0:
      Successfully uninstalled tensorboard-2.5.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.5.0
    Uninstalling tensorflow-2.5.0:
      Successfully uninstalled tensorflow-2.5.0
ERROR: Aft

In [4]:
# plotly standard imports
import plotly.graph_objs as go
import chart_studio.plotly as py

# Cufflinks wrapper on plotly
import cufflinks

# Data science imports
import pandas as pd
import numpy as np

# Options for pandas
pd.options.display.max_columns = 30

# Display all cell outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

from plotly.offline import iplot, init_notebook_mode
cufflinks.go_offline(connected=True)
init_notebook_mode(connected=True)

# Set global theme
cufflinks.set_config_file(world_readable=True, theme='pearl')

## Save simple index

Save simple index without compression

In [5]:
import numpy as np

num_rows = 17553713
batch_size = 128

filename = './indexes/wiki40b_passages_reps_32_l-8_h-768_b-512-512.dat'

passages_reps = np.memmap(
    filename,
    dtype='float32', mode='r',
    shape=(num_rows, batch_size)
)

# Train index with compression

In [7]:
import scann

# Based on https://github.com/google-research/google-research/blob/master/scann/docs/example.ipynb
num_leaves=262144
num_leaves_to_search=num_leaves // 1
training_sample_size=17553713

searcher = scann.scann_ops_pybind.builder(passages_reps, 10, "dot_product") \
    .tree(
        num_leaves=num_leaves, 
        num_leaves_to_search=num_leaves_to_search,
        training_sample_size=training_sample_size,
    ) \
    .score_brute_force(quantize=True) \
    .build()


In [9]:
save_index_filename = './indexes/scann262144x1_wiki40b_num_17553713_brute_force/'

In [11]:
import os

os.makedirs(save_index_filename, exist_ok=True)

searcher.serialize(save_index_filename)